In [1]:
import boto3
import os
from langchain.llms.bedrock import Bedrock
from langchain.retrievers import AmazonKendraRetriever
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.agents import load_tools,Tool
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory
from langchain.agents import initialize_agent, AgentType


In [2]:
dynamodb_resource=boto3.resource('dynamodb')
boto3_bedrock = boto3.client("bedrock-runtime")
kendra_client=boto3.client(service_name='kendra')

In [3]:
table_name = "ReInventAgentStack-TblSessionTableAA3432F8-1G5FPAT3WCKV" #os.environ.get('TABLE_SESSION')
table_name_agenda = "ReInventAgentStack-TblReInventAgendaB6AA89B7-R3IV15645HXF" #os.environ.get('TABLE_NAME')
kendra_index_id = "a0a1e294-931e-410e-9f64-e01cef87e58c" #os.environ.get('KENDRA_INDEX')
lambda_query_function_name = "ReInventAgentStack-FndynamodbqueryC2D26368-26x2eia2AY7x" #os.environ.get('LAMBDA_QUERY_NAME')
bedrock_model_id = "anthropic.claude-instant-v1" #os.environ.get('MODEL_ID')
model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000}



In [4]:
llm = Bedrock(model_id=bedrock_model_id, model_kwargs=model_parameter,client=boto3_bedrock)

In [5]:
def kendra_retriever(kendra_index_id,llm):
    retriever = AmazonKendraRetriever(index_id=kendra_index_id)
    memory_kendra = ConversationBufferMemory(memory_key="chat_history", return_messages=True,ai_prefix="A",human_prefix="H")


    Kendra_prompt_template = """Human: You are an Agenda re:Invent 2023 Assistant. 
    You are talkative and provides specific details from its context.
    If the AI does not know the answer to a question, it truthfully says it 
    does not know.

    Assistant: OK, got it, I'll be a talkative truthful AI assistant.

    Human: Here are a few documents in <documents> tags:
    <documents>
    {context}
    </documents>
    Based on previous documents, provide a list of data for each documents (If you find more than one), example: id, speaker, veneu, date... , it's okay if you respond with just one option, the information is important so I can decide the best choise for me to: {question}

    Assistant:
    """
    PROMPT = PromptTemplate(
        template=Kendra_prompt_template, input_variables=["context", "question"]
    )

    condense_qa_template_kendra = """{chat_history}
    Human:
    Given the previous conversation and a follow up question below, based on previous documents, provide a list of data for each document (If you find more than one), example: id, speaker, veneu, date... , it's okay if you respond with just one option, the information is important so I can decide the best choise for me.
    Follow Up Question: {question}
    Standalone Question:

    Assistant:"""
    standalone_question_prompt_kendra = PromptTemplate.from_template(condense_qa_template_kendra)

    qa = ConversationalRetrievalChain.from_llm(
        llm=llm, 
        retriever=retriever, 
        condense_question_prompt=standalone_question_prompt_kendra, 
        return_source_documents=False, 
        combine_docs_chain_kwargs={"prompt":PROMPT},
        memory = memory_kendra,
        #verbose=True
        )

    return qa

In [6]:
def define_kendra_tools(qa,tools):
    tools.append(
    Tool.from_function(
        func=qa.run,
        name="re-invent-agenda-2023",
        description="useful when you want search sessions in re:Invent 2023 agenda. This will output documentation in text type, and /n, you must deliver a coherent and accurate response from all the documentation provided. ",
    )
    )
    return tools

In [7]:
def promp_definition():

    prompt_template = """
        You are an assistant who provides recommendations on re:invent 2023 sessions, you deliver the recommendation according to their question, and also do casual conversation. 
        Use the following format:
        History: the context of a previous conversation with the user. Useful if you need to recall past conversation, make a summary, or rephrase the answers. if History is empty it continues.
        Question: the input question you must answer
        Thought: you should always think about what to do, Also try to follow steps mentioned above.
        Action: the action to take, should be one of ['re-invent-agenda-2023',"search-session-id-information"], provides options that have the most information that can be useful in making a decision.
        Action Input: the input to the action
        Observation: the result of the action
        Thought: I now know the final answer
        Final Answer: the final answer to the original input question, If you have more than one answer, give them all. Always reply in the original user language and human legible.

        History: 
        {chat_history}

        Question: {input}

        Assistant:
        {agent_scratchpad}"""

    updated_prompt = PromptTemplate(
    input_variables=['chat_history','input', 'agent_scratchpad'], template=prompt_template)

    return updated_prompt

In [8]:
def memory_dynamodb(id,table_name_session,llm):
    message_history = DynamoDBChatMessageHistory(table_name=table_name_session, session_id=id)
    memory = ConversationBufferMemory(
        memory_key="chat_history", llm=llm,max_token_limit=800,chat_memory=message_history, return_messages=True,ai_prefix="A",human_prefix="H"
    )
    return memory

In [9]:
def langchain_agent(memory,tools,llm):
    zero_shot_agent=initialize_agent(
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    #verbose=True,
    max_iteration=1,
    #return_intermediate_steps=True,
    handle_parsing_errors=True,
    memory=memory
)
    return zero_shot_agent

# Run the solution

In [10]:
def qa_to_an_agenda (prompt,session_id):

    tools = load_tools(
                        ["awslambda"],
                        awslambda_tool_name="search-session-id-information",
                        awslambda_tool_description="useful for searching session information data by their ID number, only send the ID number",
                        function_name=lambda_query_function_name,
                    )

    qa = kendra_retriever(kendra_index_id,llm)
    tools = define_kendra_tools(qa,tools)
    memory  = memory_dynamodb(session_id,table_name,llm)
    agent = langchain_agent(memory,tools,llm)
    agent.agent.llm_chain.prompt=promp_definition()
    completion = agent(prompt)

    print(completion['output'])

In [11]:
prompt = "I want know about the session with id BOA212"
session_id = "100"

In [12]:
qa_to_an_agenda (prompt,session_id)

I apologize, but as an AI assistant without direct access to the re:Invent 2023 agenda, I do not have information about individual sessions by ID code. Some topics covered in generative AI sessions may include GANs, text generation, image generation, music/video generation, and applications of generative AI. You could try searching online for "re:Invent 2023 session BOA212" or checking the AWS website for any publicly available details on sessions of interest. Please let me know if you have any other questions!


In [13]:
prompt = "I want know about generative ai session"
session_id = "1"

In [14]:
qa_to_an_agenda (prompt,session_id)

Unfortunately I don't have access to the actual re:Invent 2023 agenda or session information to look up specifics. Generative AI generally refers to machine learning techniques that can generate new data, such as text, images, audio and video. Some sessions that may be relevant at re:Invent and cover generative AI topics could include machine learning, deep learning, natural language processing (NLP), computer vision, generative adversarial networks (GANs), and reinforcement learning as those techniques are commonly used for generative applications. I recommend checking the official re:Invent website when the agenda is released for sessions related to those topics if you're interested in learning more about generative AI techniques.


In [15]:
!aws s3 ls

2023-02-22 16:25:15 528681203296-us-west-2-personalize-anime-dataset-2
2022-11-14 15:25:16 528681203296-us-west-2-personalize-anime-dataset1
2023-03-01 16:26:12 amplify-amplifyc1095e5dc28e4-staging-192605-deployment
2023-03-06 10:33:24 amplify-amplifyefe826c08deb4-staging-133318-deployment
2022-11-16 22:18:31 amplify-apppersonalize-dev-221819-deployment
2022-11-16 23:21:45 amplify-apppersonalize-dev-232135-deployment
2023-03-06 15:31:28 amplify-apprecomendador-dev-183120-deployment
2023-03-06 19:51:08 amplify-apprecomendador-dev-225059-deployment
2023-02-05 20:42:07 amplify-apprecomendador-dev-234158-deployment
2023-02-10 19:25:30 amplify-awsapprecomendadoran-staging-222523-deployment
2023-05-04 14:49:40 apistack-cwworkshopimagesf0da5e9f-1m7xl04szkf1y
2023-07-30 18:53:17 aws-athena-query-results-us-east-1-528681203296
2023-03-28 10:07:32 aws-cloudtrail-logs-528681203296-c7af33c7
2023-08-06 18:55:16 aws-glue-assets-528681203296-us-east-1
2023-09-13 17:18:52 awschatbottranslatorstack-fil

In [ ]:
prompt = "Did you know a similar session?"
session_id = "1"

In [ ]:
qa_to_an_agenda (prompt,session_id)